In [13]:
import torch
from torch.utils.data import Dataset, DataLoader
import unicodedata
import string

# in general, we cannot have all the unicode characters encoded..

class TextDataset(Dataset):

    SPECIAL_CHARS = [None, "\n", "β"]
    def __len__(self):
        return len(self.total_examples)
        pass
     
    # Makes the vocab from the given dataset
    def make_vocab(self):
        
        self.char2index = {}
        self.index2char = {}
        index = 0
        with open(self.text_file) as file:
            for line in file:
                for char in line:
                    if char not in self.char2index:
                        self.char2index[char] = index
                        self.index2char[index]  = char
                        index+=1
        
        for char in self.SPECIAL_CHARS:
#             print("this is the char")
            if char not in self.char2index:
                self.char2index[char] = index
                self.index2char[index] = char
                index += 1
        
        print(self.char2index[None])
        self.vocab_size = len(self.char2index)
    
    # this allows the model to handle all possible ASCII (non-unicode) strings passed into it!!
    def make_full_vocab(self):
        self.char2index = {}
        self.index2char = {}
        index = 0
        all_letters = string.printable 
        for char in all_letters:
            if char not in self.char2index:
                self.char2index[char] = index
                self.index2char[index]  = char
                index+=1
                
        for char in self.SPECIAL_CHARS:
            if char not in self.char2index:
                self.char2index[char] = index
                self.index2char[index] = char
                index += 1
                
        self.vocab_size = len(self.char2index)   
        
    def generate_tensor_for_char(self, char):
        temp = torch.zeros(1, self.vocab_size)
        temp[0][self.char2index[char]] = 1 
        return temp

    def lineToTensor(self,line):
        tensor = torch.zeros(len(line), 1, self.vocab_size)
        for li, letter in enumerate(line):
            tensor[li][0][self.char2index[letter]] = 1
        return tensor
      
    def get_None_tensor(self):
        tensor = torch.zeros(1, 1, self.vocab_size)
        tensor[0][0][self.char2index[None]] = 1
        return tensor
    
    def list_to_tensor(self, input_list):
        tensor = torch.zeros(len(input_list), 1, self.vocab_size)
        for elt in input_list:
            tensor[li][0][self.char2index[letter]] = 1
    
    # Get the tensor representing a new line character        
    def get_new_line_tensor(self):
        tensor = torch.zeros(1, 1, self.vocab_size)
        tensor[0][0][self.char2index["\n"]] = 1
        return tensor

    def unicodeToAscii(self,s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
            and c in self.char2index
        )

    def __init__(self, text_file,  convert_to_ascii = True):
        self.text_file = text_file

        self.training_examples = []

        total_inputs = []
        total_outputs = []
        
#         make the vocab
        self.make_full_vocab()
        
        with open(text_file) as file:
            for raw_line in file:
                
                line = self.unicodeToAscii(raw_line) if convert_to_ascii else raw_line

                inputs = self.lineToTensor([None] + [x for x in line])
                
                targets = self.lineToTensor([x for x in line] + ["\n"])  # we need a 0 as well!

                total_inputs.append(inputs)
                total_outputs.append(targets)

        assert len(total_inputs) == len(total_outputs)
        
        self.total_examples = list(zip(total_inputs, total_outputs))
        
    def __getitem__(self, index):
        return self.total_examples[index]


In [14]:
# ok, so now we can start building the network that will process these!!
# simply: iterate the dataset, and run train on it, do the log likelihood and etc.

import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)
    


In [15]:




# returns the loss for a line, evaluating on a BASIC rnn, and using the provided optimizer
def prototype_general_train(input_tensor, target_tensor, basic_rnn, optimizer):
    
        
    
    optimizer.zero_grad()
    
    criterion = nn.NLLLoss()    
    hidden = basic_rnn.initHidden()
    loss = 0
    basic_rnn.zero_grad()
    for i in range(input_tensor.size()[0]):
        output, hidden = basic_rnn(input_tensor[i], hidden)
        
        max_tensor = torch.argmax(target_tensor[i])
        ind = (torch.argmax(target_tensor[i])).item()
        true_target_tensor = torch.tensor([ind], dtype=torch.long)
        #     Evaluate the loss on each character!
        loss += criterion(output, true_target_tensor)
        
    loss.backward()
    optimizer.step()

    return output, loss.item()




In [16]:


def prototype_general_train_function(dataset):
# del rnn
# rnn = RNN(abc.vocab_size, n_hidden, abc.vocab_size)

    epoch_loss = 0
    # loss every k iters
    # 
    loss_per_k = 0
    
    total_loss = 0
    total_length  = 0
    epoch_length = 0


    num_epochs = 10
    
    
    rnn = RNN(dataset.vocab_size, 512, dataset.vocab_size)
    # If you set this too high, it might explode. If too low, it might not learn
    #  we had to set it a big lower to force convergence
    import torch.optim as optim
    optimizer = optim.SGD(rnn.parameters(), lr = 0.0001)
    
    for k in range(num_epochs ):

        for i,(x,y) in enumerate(tqdm(dataset)):
        #     print(x)
        #     print(x)
        #     print(y)
            _, loss = prototype_general_train(x,y, rnn, optimizer)
            epoch_loss += loss
            epoch_length += x.size()[0]
            total_loss += loss
            total_length += x.size()[0]
            loss_per_k += loss
        #     print(i)
        #     abc = i
        
            # for the first 100 iters, print the loss of every line!
#             if i < 100:
#                 print ("loss for {} is {}".format(i,epoch_loss/(i+1)))
                
            
            if i % 100 == 0 and i != 0:
                print ("loss for {} is {}".format(i,loss_per_k/100))
                loss_per_k = 0
                

        print("epoch {} loss is {}".format(k, epoch_loss/i))
        print("per character loss is {}".format(epoch_loss/epoch_length))
        epoch_loss = 0
        epoch_length = 0

In [13]:

from tqdm import tqdm
# make a new dataset
final_reuters_dataset = TextDataset( "reuters_news_10000.txt", True)
prototype_general_train_function(final_reuters_dataset)


  1%|▊                                                                              | 97/10000 [00:13<26:27,  6.24it/s]

loss for 100 is 252.1782168006897


  2%|█▌                                                                            | 200/10000 [00:26<28:36,  5.71it/s]

loss for 200 is 234.55296186447143


  3%|██▎                                                                           | 300/10000 [00:38<25:40,  6.30it/s]

...

loss for 4800 is 156.28145455360414


 49%|█████████████████████████████████████▋                                       | 4899/10000 [09:42<09:22,  9.06it/s]

loss for 4900 is 142.72466366767884


 50%|██████████████████████████████████████▍                                      | 4991/10000 [09:53<17:32,  4.76it/s]

KeyboardInterrupt: 

In [20]:
print("β-thal")
s = "β-thal"

β-thal


In [17]:
def unicodeToAscii(s):
    return ''.join(
        
        c for c in unicodedata.normalize('NFD', s) # take all characters from this result, then check the oboolean conditions on em
        if unicodedata.category(c) != 'Mn' #mark, nons-spacing
        and c in string.printable
    )

# we want something a bit more brusque for example:
# we literally, want to replace all characters in the text with a β character




# we should make an UNK token
# the unk token will be something like a special tensor 
# now, 

In [18]:

# unicode to regular text, but marks the text with regular beta symbols
# ideally, we can write this in a vectorized way, as opposed to just using a for loop
#  i mean, we could write a lambda for it; or we could write a list comprehension
def unicodeToMarkedAscii(s, mark="β"):
#     return "".join([c for c in s if c in unicodedata.normalize('NFD', s) and c in string.printable 
#                     else mark])
    return "".join([c if c in  unicodedata.normalize('NFD', s) and c in string.printable  else mark for c in s])



In [19]:
unicodeToMarkedAscii("john")

'john'

In [20]:

my_eqn = """
p=Exp∑BiXi / (1+Exp∑BiXi), multivariate logistic regression predictive model that calculated the risk of postoperative morbidity was developed, p = 1/(1 + e((4.810-1.287X1-0.504X2-0.500X3-0.474X4-0.405X5-0.318X6-0.316X7-0.305X8-0.278X9-0.255X10-0.138X11))).
"""

In [21]:
unicodeToMarkedAscii(my_eqn )

'\np=ExpβBiXi / (1+ExpβBiXi), multivariate logistic regression predictive model that calculated the risk of postoperative morbidity was developed, p = 1/(1 + e((4.810-1.287X1-0.504X2-0.500X3-0.474X4-0.405X5-0.318X6-0.316X7-0.305X8-0.278X9-0.255X10-0.138X11))).\n'

In [22]:

# returns the loss for the entire line (requires normalization for comparison)
def get_loss_on_line(rnn, line, vocab):
    input_tensor =  vocab.lineToTensor([None] + [x for x in line])
    target_tensor = vocab.lineToTensor([x for x in line] + ["\n"])
    criterion = nn.NLLLoss()    
    hidden = rnn.initHidden()
    loss = 0
    rnn.zero_grad()
    
    
#     print(input_tensor.size())
#     print(target_tensor[0])
#     print(target_tensor)
    for i in range(input_tensor.size()[0]):
        output, hidden = rnn(input_tensor[i], hidden)
        
        
        # had to do this simply because of how the NLLL and CrossEntropy are defined
        max_tensor = torch.argmax(target_tensor[i])
        ind = (torch.argmax(target_tensor[i])).item()
        true_target_tensor = torch.tensor([ind], dtype=torch.long)
    
#     Evaluate the loss on each character!
        loss += criterion(output, true_target_tensor)
    
    return loss.item()



In [23]:
# ok, so this is equivalent to just stripping out the bad characters

# this function runs a test. For a given set of test examples, it will create a dataset, and 
# then run the model on each of them, and then return the most confusing sentences!!

THRESHOLD = 2 



def TestHarness(test_examples, eval_model): 
    
    sent_values = []
# 
    for line in tqdm(test_examples):

        example_loss = get_loss_on_line(eval_model, unicodeToMarkedAscii(line))/len(line )
    #     print(example_loss)
        sent_values.append((line, example_loss))
        if (example_loss > THRESHOLD):
    #         print(line)
            pass
        pass

    # return the k highest lines
    most_confusing  = sorted(sent_values, key=lambda x: x[1], reverse=True)
    for (sent, val) in most_confusing[:100]:
        print("{} had loss of {}\n".format(sent, val))
    return most_confusing

    
    pass